In [94]:
import numpy as np
import math, time

In [92]:
rows, cols = 30, max(8, 4) + 2
Tab_OCW = np.zeros((rows, cols), dtype=float)
Tab_OCW[:,0] = np.linspace(0, 512, rows)  # STA 수 축
for c in range(1, cols):
    print(8.0 ** (2**(c - 1)))
    Tab_OCW[:,c] = 8.0 * (2**(c - 1))       # 예시값: 8,16,32,...

8.0
64.0
4096.0
16777216.0
281474976710656.0
7.922816251426434e+28
6.277101735386681e+57
3.940200619639448e+115
1.552518092300709e+231


In [111]:
i, start_t = 1, time.time()
rng = np.random.default_rng(0)

In [115]:
unassoc_in_number = 2
unassoc_out_number = 1

next_unassoc_in_time = 200
next_unassoc_out_time = 400

N_STA= 20
STA_Type = np.zeros(N_STA)

Mode_UORA = 0
Mode_OBO_BEB = True
Mode_OBO_XIXD = False
Opt_Random_N_RA_RU_0 = False



In [121]:
# RU/OCW 테이블
N_RU = 16
N_RA_RU_0    = 8     # assoc용 랜덤접근 RU 수
N_RA_RU_2045 = 4     # unassoc용 랜덤접근 RU 수
RU_AID = np.zeros(N_RU, dtype=int)   # 0 또는 2045 등 AID 맵 (필요시 수정)

In [120]:
# CW/alpha
CWmin, CWmax = 8.0, 1024.0
Init_alpha, Min_alpha, Max_alpha = 1.0, 0.05, 4.0
Inc, Dec = 0.10, 0.10
alpha_g = 1.0  # Mode 2/3용 글로벌 알파

In [114]:
L_MPDU              = np.full(N_STA, 2000.0)
MCS                 = np.full(N_STA, 5, dtype=int)
STA_CWmin           = np.full(N_STA, 8.0, dtype=float)
STA_CWmax           = np.full(N_STA, 1024.0, dtype=float)
STA_CW              = np.full(N_STA, 8.0, dtype=float)
STA_OBO             = np.floor(STA_CW * rng.random(N_STA))
STA_Assigned_RU     = np.zeros(N_STA, dtype=int)
STA_Assoc_Delay     = np.zeros((N_STA, 3), dtype=float)
n_tf                = np.zeros(N_STA, dtype=float)

In [124]:
Sim_Rst = np.zeros((N_STA,4), dtype=float) 

In [ ]:
while i <= 20000 :
    if next_unassoc_in_time <= i and unassoc_in_number > 0 :
        STA_ID_disable = np.where(STA_Type == -1)[0]
        if len(STA_ID_disable) == unassoc_in_number :
            STA_Type[STA_ID_disable] = 0
            next_unassoc_in_time = i + 500
            print('1')
        else :
            print('2')
            old_N = N_STA
            N_STA += int(unassoc_in_number)
            next_unassoc_in_time = i + 500
            
            STA_Type            = np.concatenate([STA_Type, np.zeros(int(unassoc_in_number), dtype=int)])
            L_MPDU              = np.concatenate([L_MPDU, np.full(int(unassoc_in_number), 2000.0)])
            MCS                 = np.concatenate([MCS, np.full(int(unassoc_in_number), 5, dtype=int)])
            STA_CWmin           = np.concatenate([STA_CWmin, np.full(int(unassoc_in_number), 8.0, dtype=float)])
            STA_CWmax           = np.concatenate([STA_CWmax, np.full(int(unassoc_in_number), 1024.0, dtype=float)])
            STA_CW              = np.concatenate([STA_CW, np.full(int(unassoc_in_number), 8.0, dtype=float)])
            STA_OBO             = np.concatenate([STA_OBO, np.floor(8.0 * rng.random(int(unassoc_in_number)))])
            STA_Assigned_RU     = np.concatenate([STA_Assigned_RU, np.zeros(int(unassoc_in_number), dtype=int)])
            n_tf                = np.concatenate([n_tf, np.zeros(int(unassoc_in_number), dtype=float)])
            Sim_Rst             = np.vstack([Sim_Rst, np.zeros((int(unassoc_in_number), 4), dtype=float)])
            add_delay = np.zeros((int(unassoc_in_number), 3), dtype=float)
            add_delay[:, 0] = (next_unassoc_in_time - 500)

            if Mode_UORA in (4, 5, 6, 7, 8) :
                alpha = np.concatenate([alpha, np.full(int(unassoc_in_number), 1.0, dtype=float)])
                if Mode_UORA in (5, 6, 7, 8) :
                    Succ = np.vstack([Succ, np.zeros((int(unassoc_in_number), 2), dtype=float)])
        
    if next_unassoc_out_time <= i and unassoc_out_number > 0 :
        print('3')
        STA_ID_Assoc = np.where(STA_Type == 1)[0]
        Disable_STA_ID = []
        while len(Disable_STA_ID) < int(unassoc_out_number) and len(STA_ID_Assoc) > 0:
            pick = rng.choice(STA_ID_Assoc)
            if pick not in Disable_STA_ID:
                Disable_STA_ID.append(pick)
                STA_Type[pick] = -1
        next_unassoc_out_time = i + 500
    
    STA_ID_Unassoc  = np.where(STA_Type == 0)[0]
    STA_ID_Assoc    = np.where(STA_Type == 1)[0]

    if Mode_UORA in (0, 9) :
        if Opt_Random_N_RA_RU_0:
            N_Random_RA_RU_0 = int(math.ceil(rng.random() * N_RA_RU_0))
            if STA_ID_Unassoc.size:
                STA_OBO[STA_ID_Unassoc] -= N_RA_RU_2045
            if STA_ID_Assoc.size:
                STA_OBO[STA_ID_Assoc] -= N_Random_RA_RU_0
        else:
            if STA_ID_Unassoc.size:
                STA_OBO[STA_ID_Unassoc] -= N_RA_RU_2045
            if STA_ID_Assoc.all:
                STA_OBO[STA_ID_Assoc] -= N_RA_RU_0

    elif Mode_UORA == 1:
        raise RuntimeError('Mode_UORQ == 1 not supported in provided script')

    elif Mode_UORA == 2:
        STA_OBO -= alpha_g * N_RU

    elif Mode_UORA == 3:
        STA_OBO -= alpha_g * N_RU
    
    elif Mode_UORA in (4, 5, 6, 7, 8) :
        if STA_ID_Unassoc.size:
            STA_OBO[STA_ID_Unassoc] -= alpha[STA_ID_Unassoc] * N_RA_RU_2045

        if STA_ID_Assoc.size:
            if Opt_Random_N_RA_RU_0:
                NR = N_RA_RU_0
            else :
                NR = N_RA_RU_0
            STA_OBO[STA_ID_Assoc] -= alpha[STA_ID_Assoc] * NR
    i = i+1